# Preprocess NIH X-Ray Dataset

In [1]:
import pandas as pd

metadata = pd.read_csv("/home/sn666/explainable_ai/LaBo/datasets/XRAY_NIH/Data_Entry_2017.csv")

metadata

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,058Y,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,058Y,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,058Y,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,081Y,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,081Y,F,PA,2582,2991,0.143,0.143,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,039Y,M,PA,2048,2500,0.168,0.168,NaN
112116,00030802_000.png,No Finding,0,30802,029Y,M,PA,2048,2500,0.168,0.168,NaN
112117,00030803_000.png,No Finding,0,30803,042Y,F,PA,2048,2500,0.168,0.168,NaN
112118,00030804_000.png,No Finding,0,30804,030Y,F,PA,2048,2500,0.168,0.168,NaN


In [2]:
class2images = {}

for index, row in metadata.iterrows():
    image = row["Image Index"]
    for label in row["Finding Labels"].split("|"):
        class2images.setdefault(label, []).append(image)

In [6]:
class2images.keys()

dict_keys(['Cardiomegaly', 'Emphysema', 'Effusion', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Atelectasis', 'Pneumothorax', 'Pleural_Thickening', 'Pneumonia', 'Fibrosis', 'Edema', 'Consolidation'])

In [4]:
# get rid of the "No Finding" class
class2images.pop("No Finding")

['00000002_000.png',
 '00000005_000.png',
 '00000005_001.png',
 '00000005_002.png',
 '00000005_003.png',
 '00000005_004.png',
 '00000005_005.png',
 '00000006_000.png',
 '00000007_000.png',
 '00000008_001.png',
 '00000011_001.png',
 '00000011_002.png',
 '00000011_003.png',
 '00000011_004.png',
 '00000011_008.png',
 '00000013_000.png',
 '00000013_008.png',
 '00000013_014.png',
 '00000013_015.png',
 '00000013_016.png',
 '00000013_017.png',
 '00000013_019.png',
 '00000013_038.png',
 '00000014_000.png',
 '00000015_000.png',
 '00000016_000.png',
 '00000017_001.png',
 '00000017_002.png',
 '00000018_000.png',
 '00000022_000.png',
 '00000023_000.png',
 '00000023_001.png',
 '00000023_003.png',
 '00000029_000.png',
 '00000031_000.png',
 '00000032_002.png',
 '00000032_003.png',
 '00000032_005.png',
 '00000032_008.png',
 '00000032_009.png',
 '00000032_010.png',
 '00000032_019.png',
 '00000032_020.png',
 '00000032_022.png',
 '00000032_025.png',
 '00000032_042.png',
 '00000032_046.png',
 '00000032_04

In [5]:
class2images.keys()

dict_keys(['Cardiomegaly', 'Emphysema', 'Effusion', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Atelectasis', 'Pneumothorax', 'Pleural_Thickening', 'Pneumonia', 'Fibrosis', 'Edema', 'Consolidation'])

In [8]:
train_images = []
with open("/home/sn666/explainable_ai/LaBo/datasets/XRAY_NIH/train_val_list_NIH.txt") as f:
    for line in f:
        train_images.append(line.strip())

# split train_images into train and val
import random

random.seed(42)

random.shuffle(train_images)

print(len(train_images))
split = int(0.8 * len(train_images))

val_images = train_images[split:]
train_images = train_images[:split]

86524


In [9]:
test_images = []
with open("/home/sn666/explainable_ai/LaBo/datasets/XRAY_NIH/test_list_NIH.txt") as f:
    for line in f:
        test_images.append(line.strip())

len(train_images), len(val_images), len(test_images)

(69219, 17305, 25596)

In [10]:
class2images.keys()

dict_keys(['Cardiomegaly', 'Emphysema', 'Effusion', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Atelectasis', 'Pneumothorax', 'Pleural_Thickening', 'Pneumonia', 'Fibrosis', 'Edema', 'Consolidation'])

In [11]:
# rename Pleural_Thickening to Pleural Thickening
class2images["Pleural Thickening"] = class2images.pop("Pleural_Thickening")

In [14]:
class2images_train = {}
class2images_val = {}
class2images_test = {}

for label, images in class2images.items():
    class2images_train[label] = [image for image in images if image in train_images]
    class2images_val[label] = [image for image in images if image in val_images]
    class2images_test[label] = [image for image in images if image in test_images]

In [15]:
len(class2images_train['Cardiomegaly'])

1342

In [19]:
class2images_train.keys()

dict_keys(['Cardiomegaly', 'Emphysema', 'Effusion', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Atelectasis', 'Pneumothorax', 'Pneumonia', 'Fibrosis', 'Edema', 'Consolidation', 'Pleural Thickening'])

In [35]:
len(class2images_val['Cardiomegaly'])

365

In [16]:
class2images_val.keys()

dict_keys(['Cardiomegaly', 'Emphysema', 'Effusion', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Atelectasis', 'Pneumothorax', 'Pneumonia', 'Fibrosis', 'Edema', 'Consolidation', 'Pleural Thickening'])

In [17]:
len(class2images_test['Cardiomegaly'])

1065

In [18]:
splits_path = "/home/sn666/explainable_ai/LaBo/datasets/XRAY_NIH/splits"

import os
import pickle

with open(os.path.join(splits_path, "class2images_train.p"), "wb") as f:
    pickle.dump(class2images_train, f)

with open(os.path.join(splits_path, "class2images_val.p"), "wb") as f:
    pickle.dump(class2images_val, f)

with open(os.path.join(splits_path, "class2images_test.p"), "wb") as f:
    pickle.dump(class2images_test, f)

In [1]:
import numpy as np

In [3]:
# read /home/sn666/explainable_ai/LaBo/exp/asso_opt/XRAY_NIH/XRAY_NIH_allshot_fac/concept2cls_selected.npy
concept2cls_selected = np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/XRAY_NIH/XRAY_NIH_allshot_fac/concept2cls_selected.npy", allow_pickle=True)
concept2cls_selected

array([0., 0.])

In [7]:
import torch

select_idx = torch.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/XRAY_NIH/XRAY_NIH_allshot_fac/select_idx.pth")
select_idx

tensor([0, 1])

In [8]:
concept2cls_selected = np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000/HAM10000_allshot_fac/concept2cls_selected.npy", allow_pickle=True)
concept2cls_selected

array([3., 3., 5., ..., 4., 0., 0.])

In [5]:
import torch

tens = torch.tensor(
    [
        [
            0.0173,
            0.0176,
            0.0186,
            0.0179,
            0.0187,
            0.0172,
            0.0178,
            0.0174,
            0.0169,
            0.0181,
            0.0181,
            0.0176,
            0.0183,
            0.0174,
        ],
        [
            0.0222,
            0.0221,
            0.0231,
            0.0226,
            0.0229,
            0.0220,
            0.0224,
            0.0222,
            0.0221,
            0.0226,
            0.0222,
            0.0222,
            0.0230,
            0.0223,
        ],
        [
            0.0208,
            0.0207,
            0.0211,
            0.0207,
            0.0222,
            0.0201,
            0.0205,
            0.0204,
            0.0197,
            0.0207,
            0.0205,
            0.0206,
            0.0209,
            0.0206,
        ],
        [
            0.0204,
            0.0203,
            0.0211,
            0.0210,
            0.0212,
            0.0201,
            0.0204,
            0.0203,
            0.0202,
            0.0205,
            0.0202,
            0.0204,
            0.0211,
            0.0204,
        ],
        [
            0.0326,
            0.0329,
            0.0328,
            0.0327,
            0.0327,
            0.0328,
            0.0327,
            0.0329,
            0.0330,
            0.0327,
            0.0326,
            0.0327,
            0.0327,
            0.0328,
        ],
    ]
)

tens

tensor([[0.0173, 0.0176, 0.0186, 0.0179, 0.0187, 0.0172, 0.0178, 0.0174, 0.0169,
         0.0181, 0.0181, 0.0176, 0.0183, 0.0174],
        [0.0222, 0.0221, 0.0231, 0.0226, 0.0229, 0.0220, 0.0224, 0.0222, 0.0221,
         0.0226, 0.0222, 0.0222, 0.0230, 0.0223],
        [0.0208, 0.0207, 0.0211, 0.0207, 0.0222, 0.0201, 0.0205, 0.0204, 0.0197,
         0.0207, 0.0205, 0.0206, 0.0209, 0.0206],
        [0.0204, 0.0203, 0.0211, 0.0210, 0.0212, 0.0201, 0.0204, 0.0203, 0.0202,
         0.0205, 0.0202, 0.0204, 0.0211, 0.0204],
        [0.0326, 0.0329, 0.0328, 0.0327, 0.0327, 0.0328, 0.0327, 0.0329, 0.0330,
         0.0327, 0.0326, 0.0327, 0.0327, 0.0328]])

In [6]:
torch.sigmoid(tens) > 0.5

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True]])

In [10]:
import numpy as np
t1 = np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000/HAM10000_allshot_fac_biomedclip/concepts_raw_selected.npy")

In [11]:
t2 = np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000/HAM10000_allshot_fac/concepts_raw_selected.npy")

In [13]:
(t1 == t2).all()

True

In [3]:
import numpy as np

np.load("/home/sn666/explainable_ai/LaBo/datasets/XRAY_NIH/concepts/concepts_raw.npy")

array(['Elevation of the hemidiaphragm on the affected side',
       'Displacement of Hilar Structures: The hilar structures might be displaced towards the affected side.',
       'Decreased rib expansion on affected side', ...,
       'Aberrant gastric air pattern',
       'Presence of soft tissue mass in unusual locations',
       'Unilateral Diaphragmatic Elevation'], dtype='<U186')

In [1]:
import json

In [5]:
import os

with open("/home/sn666/explainable_ai/LaBo/datasets/HAM10000/concepts/class2concepts.json", 'r') as f:
    ham10000_concept = json.load(f)

ham10000_concept.keys()

dict_keys(['actinic keratoses', 'basal cell carcinoma', 'benign keratosis-like lesions', 'dermatofibroma', 'melanocytic nevi', 'melanoma', 'vascular lesions'])

In [2]:
import numpy as np

np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000/HAM10000_specialist_allshot_fac/concepts_raw_selected.npy")

array(['Absence of atypical network',
       'Absence of atypical vascular patterns',
       'Absence of atypical vasculature: Nevi usually do not show unusual blood vessels or structures like those seen in more concerning lesions.',
       ..., 'Yellowish undertones', 'Yellowish-tan color',
       'Yellowness or yellow hue'], dtype='<U231')

In [4]:
np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000/HAM10000_specialist_allshot_fac/concepts_raw_selected.npy")

array(['Absence of atypical network',
       'Absence of atypical vascular patterns',
       'Absence of atypical vasculature: Nevi usually do not show unusual blood vessels or structures like those seen in more concerning lesions.',
       ..., 'Yellowish undertones', 'Yellowish-tan color',
       'Yellowness or yellow hue'], dtype='<U231')

In [6]:
import os
import platform

def creation_date(path_to_file):
    """
    Try to get the date that a file was created, falling back to when it was
    last modified if that isn't possible.
    See http://stackoverflow.com/a/39501288/1709587 for explanation.
    """
    if platform.system() == 'Windows':
        return os.path.getctime(path_to_file)
    else:
        stat = os.stat(path_to_file)
        try:
            return stat.st_birthtime
        except AttributeError:
            # We're probably on Linux. No easy way to get creation dates here,
            # so we'll settle for when its content was last modified.
            mtime = stat.st_mtime
            # convert mtime to datetime
            import datetime
            return datetime.datetime.fromtimestamp(mtime)

creation_date("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000/HAM10000_specialist_allshot_fac/select_idx.pth")

datetime.datetime(2025, 3, 23, 14, 43, 30, 952196)

In [8]:
import torch
torch.load("/home/sn666/explainable_ai/LaBo/datasets/HAM10000/splits/label_train_all.pth")

tensor([0, 0, 0,  ..., 6, 6, 6])